## 1. Dask DataFrames

### Set up the Dask Client

[Dask Client API Reference](https://distributed.dask.org/en/stable/api.html#distributed.Client)  
[Dask Dataframe Tutorial](https://tutorial.dask.org/04_dataframe.html)

* These settings took a bit of trial and error to avoid a mountain of memory leak messages!
* My machine (Mac OSX) has 16 cores and 16MB of RAM.
* See the 5. Dask Client Notebook for more details

In [ ]:
from dask.distributed import Client

client = Client(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="4 GiB"
)

client

#### Making the flights.csv dataframe 20 x bigger (Pandas)

To allow for a more meaningful comparison, this concatanates the flights dataset 20 times, resulting in around 116m rows.

In [ ]:
%%time

import pandas as pd

# Specify columns to keep
usecols = [
    "YEAR", "MONTH", "DAY", "FLIGHT_NUMBER", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY",
    "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"
]

# Read the data in from csv
df = pd.read_csv("./data/flights/flights.csv", usecols=usecols)

df_20 = df.copy()

i = 1
while i < 20:
    df_20 = pd.concat([df_20, df])
    i += 1

df_20.to_csv("./data/flights/flights_limited_x20.csv", index=False)

#### Making the flights.csv dataframe 20 x bigger (Dask)

You can also do it in Dask, which is a lot quicker!

In [ ]:
%%time

import dask.dataframe as dd

# Specify columns to keep
usecols = [
    "YEAR", "MONTH", "DAY", "FLIGHT_NUMBER", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY",
    "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"
]

# Read the data in from csv
df = dd.read_csv("./data/flights/flights.csv", usecols=usecols)

df_20 = df.copy()

i = 1
while i < 20:
    df_20 = dd.concat([df_20, df])
    i = i+1

# Write to CSV
df_20.to_csv("./data/flights/flights_limited_x20_csv", index=False)

# Write to JSON
df_20.to_json(
    filename="./data/flights/flights_limited_x20_json",
    orient="records"
)

#### Pandas Speed Test

Testing the speed of some commonly used pandas functionality on our DataFrame, including:

* Reading a CSV
* Dropping na values
* Converting to Datetime
* Vectorised (across columns) sum
* Dropping
* Filtering
* Groupby / sum

In [ ]:
%%time

import pandas as pd
import numpy as np

df_20 = pd.read_csv("./data/flights/flights_limited_x20.csv")

# Fill NA values
df_20 = df_20.fillna(0)

# Create a Date column
df_20["DATE"] = pd.to_datetime(df_20[["YEAR", "MONTH", "DAY"]])

df_20["TOTAL_DELAY"] = (
    df_20[['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']].sum(axis=1)
)

# Drop Columns
df_20 = df_20.drop(["YEAR", "MONTH", "DAY"], axis=1)

# Perform Boolean Indexing (Where/Filter)
df_20 = df_20[df_20["DATE"] > pd.to_datetime("2015-01-01")]

print(df_20.shape)

# Perform an aggregation
tab = df_20.groupby(["DATE", "FLIGHT_NUMBER"]).sum()

# Show the table
tab

#### Dask Speed Test

The same functionality as the Pandas speed test, but in Dask. Note:
* Replacing pd with a reference to `dask.dataframe`
* Computing the code with `.compute()`. This converts an out-of-memory Dask DataFrame to an in-memory Pandas DataFrame.
* Note that we call compute on `tab` and not `df_20`. `df_20` will still be processed as a dependency but it won't be stored in memory.
* Ideally you should call compute just once on your final output (or not at all if writing data out).

* A good Dask workflow is:
    * Import 
    * Reduce (e.g. remove columns, filter data, remove NaN / outliers etc.)
    * Transform (e.g. casting, new columns, functions etc.)
    * Aggregate
    * Compute / Write out

In [ ]:
%%time

import dask.dataframe as dd
import pandas as pd

df_20 = dd.read_csv("./data/flights/flights_limited_x20.csv")

# Fill NA values
df_20 = df_20.fillna(0)

# Create a Date column
df_20["DATE"] = dd.to_datetime(df_20[["YEAR", "MONTH", "DAY"]])

df_20["TOTAL_DELAY"] = (
    df_20[['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']].sum(axis=1)
)

# Drop Columns
df_20 = df_20.drop(["YEAR", "MONTH", "DAY"], axis=1)

# Perform Boolean Indexing (Where/Filter)
df_20 = df_20[df_20["DATE"] > pd.to_datetime("2015-01-01")]

print(df_20.shape)

# Perform an aggregation
tab = df_20.groupby(["DATE", "FLIGHT_NUMBER"]).sum()


tab_out = tab.compute()
tab_out

In [ ]:
tab

#### Compute two ways...

In [ ]:
from dask.dataframe import compute

tab_out = compute(tab)

or...

In [ ]:
tab_out = tab.compute()

Both do the same thing.

In [ ]:
# Return an un-computed DF
tab

In [ ]:
# Return the computed DF
tab_out

In [ ]:
# Compute the first 5 rows of an un-computed DF
df_20.head(5)